In [ ]:
import PyPDF2
from dotenv import load_dotenv
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

results = []
with open('meeting/회의록.pdf', "rb") as f:
    pdf_reader = PyPDF2.PdfReader(f)
    for page_num in range(len(pdf_reader.pages)) :
        page = pdf_reader.pages[page_num]
        text = page.extract_text()
        print(f"Page : {page_num} / Text: {text} \n")
        results.append(f"Page : {page_num} / Text: {text} \n")

proceedings
content='proceedings' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 157, 'total_tokens': 160, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BeWni7tqn18gA1s0sbYIlA1867l8g', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--b0cccc22-05b7-4096-9003-63820dba555f-0' usage_metadata={'input_tokens': 157, 'output_tokens': 3, 'total_tokens': 160, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from dotenv import load_dotenv
from IPython.display import Markdown, display
load_dotenv()

# 1. 메시지 템플릿 생성
system_message = SystemMessagePromptTemplate.from_template(
    "너는 어시스턴트야. 내가 회의록 PDF 파일을 주면, 첫번째로 회의 주제 / 목적, "
    "두번째로 핵심 논의 사항, 세번째로 결정된 사항, 네번째로 보류되거나 추가 논의 항목을 "
    "요약해서 한국어로 알려줘."
)
human_message = HumanMessagePromptTemplate.from_template("{input}")

chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

# 2. LLM 생성
llm = ChatOpenAI(model="gpt-4o-mini")

# 3. Prompt와 LLM 결합
chatbot = chat_prompt | llm

# 4. 실행
response = chatbot.invoke({"input": results})
result = response.content
print(result)

In [ ]:
from langchain_community.document_loaders import PDFPlumberLoader
loader = PDFPlumberLoader("meeting/회의록.pdf")
import pdfplumber
with pdfplumber.open("meeting/회의록.pdf") as pdf :
    pages = pdf.pages # 페이지 리스트
    for page in pages : 
        im = page.to_image(resolution=150) # Page 객체 -> PageImage라는 객체로 반환
        im.draw_rects(page.extract_words()) # 페이지에서 추출한 단어들의 위치 좌표에 사각형(rect)을 그리는 함수입니다.

# 마지막 페이지만 화면 그대로 출력
im.show()

In [3]:
import os
from dataclasses import dataclass
from typing import TypedDict, Dict # Dict 추가

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.graph import END, START, StateGraph
from dotenv import load_dotenv
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
load_dotenv()
# 1-1) openai Client 생성
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("⚠️ 환경변수 OPENAI_API_KEY가 설정되지 않았습니다.")
# OpenAI 인스턴스를 만듭니다.
openai_client = OpenAI(api_key=openai_api_key)
print("✅ OpenAI 클라이언트 생성 완료")
    
# 1-2) Pinecone 인스턴스 생성
if not os.getenv("PINECONE_API_KEY") or not os.getenv("PINECONE_ENVIRONMENT"):
    raise ValueError("⚠️ 환경변수 PINECONE_API_KEY 또는 PINECONE_ENV가 누락되었습니다.")

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"), environment=os.getenv("PINECONE_ENVIRONMENT"))
print("✅ Pinecone 클라이언트 생성 완료")

# 1-3) 인덱스 존재 여부 확인
index_name = "dense-index"  # 실제 사용 중인 인덱스 이름으로 교체하세요
existing_indexes = pc.list_indexes().names()
if index_name not in existing_indexes:
    raise ValueError(f"⚠️ 인덱스 '{index_name}'가 Pinecone에 존재하지 않습니다. 현재 인덱스 목록: {existing_indexes}")

# 1-4) 해당 인덱스 객체 가져오기
index = pc.Index(index_name)
print(f"✅ Pinecone 인덱스 '{index_name}' 연결 완료 (Namespaces: {len(index.describe_index_stats().namespaces)})")

✅ OpenAI 클라이언트 생성 완료
✅ Pinecone 클라이언트 생성 완료


c:\dev\github\SKN10-FINAL-01Team\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Pinecone 인덱스 'dense-index' 연결 완료 (Namespaces: 3)


In [ ]:
import boto3
from dotenv import load_dotenv
import os
load_dotenv()
def get_s3_client() :
    return boto3.client(
        "s3",
        aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
        region_name=os.getenv('AWS_S3_REGION_NAME')
    )


bucket="code-agent"
prefix=""
s3 = get_s3_client()
resp = s3.list_objects_v2(
        Bucket=bucket,
        Prefix=prefix,
        Delimiter="/"  # 가상 폴더 단위로 끊어줌
    )

# CommonPrefixes → 한 단계 하위 폴더, Contents → 한 단계 하위 파일
folders = [p["Prefix"] for p in resp.get("CommonPrefixes", [])]
files   = [o["Key"]    for o in resp.get("Contents", []) if o["Key"] != prefix]

# 클라이언트가 쓰기 좋게 이름만 남겨서 리턴
data = [
    {"type": "folder", "name": f[len(prefix):].rstrip("/")}
    for f in folders
] + [
    {"type": "file",   "name": f[len(prefix):].rstrip("/")}
    for f in files
]
print(folders)
print(files)
print(data)

def s3_objects_api(request):
    """
    AJAX로 호출될 JSON API.
    ?bucket=이름&prefix=경로 형태로 호출.
    """
    s3 = get_s3_client()
    bucket = request.GET.get("bucket")
    prefix = request.GET.get("prefix", "")

    if not bucket:
        # 1) bucket 파라미터가 없으면 “버킷 목록” 화면
        data = [{ "type" : "folder", "name" : b["Name"]} for b in s3.list_buckets().get("Buckets", [])]
        print(data)
        return data
    
    # s3.list_objects_v2() : 버킷 안의 객체(파일) 목록을 가져오는 메서드
    # Bucket : 가져올 버킷, Prefix : 이전 경로
    # prefix="logs/2025/" 로 주면 버킷 안에서 logs/2025/ 로 시작하는 모든 파일만 조회
    resp = s3.list_objects_v2(
        Bucket=bucket,
        Prefix=prefix,
        Delimiter="/"  # 가상 폴더 단위로 끊어줌
    )

    # CommonPrefixes → 한 단계 하위 폴더, Contents → 한 단계 하위 파일
    folders = [p["Prefix"] for p in resp.get("CommonPrefixes", [])]
    files   = [o["Key"]    for o in resp.get("Contents", []) if o["Key"] != prefix]

    # 클라이언트가 쓰기 좋게 이름만 남겨서 리턴
    # 클라이언트가 쓰기 좋게 이름만 남겨서 리턴
    data = [
        {"type": "folder", "name": f[len(prefix):].rstrip("/")}
        for f in folders
    ] + [
        {"type": "file",   "name": f[len(prefix):].rstrip("/")}
        for f in files
    ]
    return data

['user_github_repos/']
[]
[{'type': 'folder', 'name': 'user_github_repos'}]
